# Validação e testes de modelos de Machine Learning

### Introdução a processos de validação mais complexos do que o hold out básico (train/test), como validação cruzada, grupos etc.

In [1]:
#carregamos os dados e expurgamos a conluna sem nome
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
#Sendo x o que queremos analisar, e y a nossa classificação
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

In [3]:
#Importando as bibliotecas para separação e treino dos modelos
#Separamos 75% para treino e 25% para teste
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [4]:
#utilizaremos a biblioteca Dummy para utilizar os dados e encontrar a Acurácia(taxa de acerto)
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


In [5]:
#utilizaremos a biblioteca Tree para utilizar os dados e encontrar a Acurácia(taxa de acerto)
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [6]:
#Utilizaremos cross validate para quebrar a estrutura em pedaços 
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=3, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accruracy com cross validation, 3 = [%.2f, %.2f]" %((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accruracy com cross validation, 3 = [74.99, 76.57]


In [7]:
#Alterando a quebra dos dados cruzados em 5
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accruracy com cross validation, 5 = [%.2f, %.2f]" %((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accruracy com cross validation, 5 = [75.21, 76.35]


# Aleatoriedade no Cross Validate

In [8]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accruracy médio %.2f" % (media * 100))
    print("Accruracy intervalo = [%.2f, %.2f]" %((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [9]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results)

Accruracy médio 75.78
Accruracy intervalo = [74.37, 77.19]


In [10]:
#Resultados aleatórios com base no shuffle, quebrando em 10 pedaços a estrutura
#metodo kfold embaralha os dados
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits=10,shuffle= True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results)

Accruracy médio 75.78
Accruracy intervalo = [73.58, 77.98]


# Simular situação horrível de azar
## Pode ser azar como pode ser uma proporção de exemplos desbalanceado entre as classe

In [11]:
#ordenando os dados vendidos por ordem crescente
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [12]:
# sem o Shaffle
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_resultados(results)

Accruracy médio 57.84
Accruracy intervalo = [34.29, 81.39]


In [13]:
# com o shaffle, mostra que temos os dados balanceado e ter uma melhor acuracia
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits=10,shuffle= True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_resultados(results)

Accruracy médio 75.76
Accruracy intervalo = [73.02, 78.50]


In [14]:
#Estratificando os dados quando se temos desbalanço entre as classes
from sklearn.model_selection import StratifiedKFold

SEED = 158020
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10,shuffle= True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, return_train_score=False)
imprime_resultados(results)

Accruracy médio 75.78
Accruracy intervalo = [74.10, 77.46]


In [15]:
#Criando coluna e adicionando dados ficticios, Coluna modelo carro
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,2
4,117384.68,1,4,12770.11290,2


In [20]:
#plotando os dados aleatórios gerados.
dados.modelo.unique()

array([17, 20, 12,  2, 10,  9, 18,  4, 16,  6,  5, 21, 15, 13, 11, 14,  8,
        3, 19,  7, 22,  0,  1, -1], dtype=int64)

In [21]:
abs(dados.modelo.min())

1

In [22]:
#ajustando os dados para não termos numero negativo.
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,4


In [26]:
#contar a quantidade de cada modelo
dados.modelo.value_counts()

20    873
19    830
18    788
21    707
17    687
15    669
16    616
22    597
14    577
13    576
12    497
11    397
10    376
23    347
9     320
8     288
7     217
24    199
6     186
5     120
4      66
3      44
2      20
1       3
Name: modelo, dtype: int64

In [27]:
#Ele aprupa os dados por grupo para validar os dados cruzados
from sklearn.model_selection import GroupKFold

SEED = 158020
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups= dados.modelo, return_train_score=False)
imprime_resultados(results)

Accruracy médio 75.76
Accruracy intervalo = [72.91, 78.60]
